In [1]:
import json
import collections
import numpy as np
import pymorphy2
analyzer = pymorphy2.MorphAnalyzer()

In [2]:
with open('dict_tokenized_corpus.json', 'r') as f:
    dict_corpus = json.load(f)

### Частеречную разметку произведем, создав новый словарь, где каждому слову в документе будет соответствовать тэг части речи

In [3]:
tagged_corpus = {}
for key, doc in dict_corpus.items():
    # нормализуем
    norm_doc = [analyzer.parse(word)[0].normal_form for word in doc]
    # проставим тэги
    tagged_doc = [analyzer.parse(word)[0].tag.POS for word in norm_doc]
    tagged_corpus[key] = tagged_doc
    # удостоверимся, что части речи отмечены правильно
    if key == '0':
        for i in range(0, 15):
            print('{0:12} {1}'.format(norm_doc[i], tagged_doc[i]))

миф          NOUN
обязательный ADJF
вибрация     NOUN
выход        NOUN
тело         NOUN
практика     NOUN
вибрация     NOUN
это          PRCL
признак      NOUN
скорый       ADJF
выход        NOUN
тело         NOUN
угодный      ADJF
выход        NOUN
осознанный   ADJF


In [4]:
print(tagged_corpus['29'])

['ADJF', 'NOUN', 'NOUN', 'ADVB', 'INFN', 'ADJF', 'NOUN', 'ADJF', 'NOUN', 'NOUN', 'INFN', 'ADJF', 'NOUN', 'NOUN']


### Создадим словарь, в котором будут в процентах подсчитаны доли разных частей речи для каждого документа

In [5]:
tags_distr = {}
for key, doc in tagged_corpus.items():
    doc_tags = collections.Counter(doc)
    doc_tags_perc = {}
    for tag, value in doc_tags.items():
        # convert to percent
        doc_tags_perc[tag] = value / sum(doc_tags.values()) * 100
    tags_distr[key] = doc_tags_perc

print(tags_distr['0'])

{'NOUN': 50.63291139240506, 'ADJF': 12.658227848101266, 'PRCL': 2.5316455696202533, 'INFN': 24.050632911392405, 'CONJ': 2.5316455696202533, 'ADVB': 6.962025316455696, 'PREP': 0.6329113924050633}


### Подсчитаем распределение и стандартное отклонение для каждой части речи во всем корпусе

In [9]:
corpus_tags = collections.defaultdict(list)
for doc in tags_distr.values():
    for tag, value in doc.items():
        corpus_tags[tag].append(value)

print(corpus_tags['NOUN'][:5])

[50.63291139240506, 46.18834080717489, 62.21198156682027, 53.84615384615385, 45.774647887323944]


In [11]:
pos_distribution = collections.defaultdict(list)
for tag, perc_list in corpus_tags.items():
    pos_distribution[tag].append(np.mean(perc_list))
    pos_distribution[tag].append(np.std(perc_list))

pos_distribution_sorted = {}
tags_sorted = sorted(pos_distribution, key=pos_distribution.get, reverse=True)
for tag in tags_sorted:
    pos_distribution_sorted[tag] = pos_distribution[tag]
for key, values in pos_distribution_sorted.items():
    print(f'{key} : {values}')

NOUN : [45.11002362646561, 11.114618234399392]
INFN : [27.908836522820327, 10.218078266819381]
ADJF : [19.722417716139038, 11.743724554754698]
ADVB : [8.579977650351763, 7.123525839565305]
PRCL : [4.5094769396925125, 5.859008752294588]
INTJ : [3.5164361014094156, 8.19786040470009]
None : [2.1133274841068785, 3.5917024733511163]
CONJ : [1.7249932123448228, 1.3257917796882164]
NPRO : [1.3957612736632243, 1.4066849721659704]
PREP : [1.3743061910727696, 3.4502832931751533]
GRND : [1.1852950873722388, 1.0369271348499836]
VERB : [1.16198501516196, 1.3765491869108488]
NUMR : [0.9240831135650585, 0.9187683987046782]
PRED : [0.794519684599948, 0.7315836659966097]
ADJS : [0.7527886119561771, 0.41706253939848786]
PRTF : [0.5573827970052156, 0.489829799539516]


### Найдем документы, в которых доля частей речи отличается более чем на два стандартных отклонения

In [12]:
odd_docs = collections.defaultdict(list)

for tag, params in pos_distribution_sorted.items():
    std_lower_bound = params[0] - params[1] * 2
    std_upper_bound = params[0] + params[1] * 2
    for key, doc in tags_distr.items():
        try:
            if not std_lower_bound < doc[tag] < std_upper_bound:
                # запишем id документа и часть речи с нетипичной долей
                odd_docs[key].append(tag)
                odd_docs[key].append(doc[tag])
        # если части речи нет в документе
        except KeyError:
            pass

In [13]:
print(len(odd_docs))
print(odd_docs)

63
defaultdict(<class 'list'>, {'14': ['NOUN', 78.57142857142857], '48': ['NOUN', 20.689655172413794, 'INFN', 58.620689655172406], '86': ['NOUN', 75.0], '117': ['NOUN', 75.0, 'ADVB', 25.0], '167': ['NOUN', 100.0], '202': ['NOUN', 20.0], '215': ['NOUN', 71.42857142857143], '230': ['NOUN', 15.151515151515152], '262': ['NOUN', 100.0], '21': ['INFN', 100.0], '37': ['INFN', 50.0], '88': ['INFN', 50.0], '110': ['INFN', 60.0, 'PRCL', 20.0], '129': ['INFN', 66.66666666666666, 'ADVB', 33.33333333333333], '168': ['INFN', 50.0, 'ADVB', 25.0], '176': ['INFN', 50.0], '244': ['INFN', 50.0, 'ADVB', 25.0], '275': ['INFN', 3.8461538461538463], '6': ['ADJF', 50.0], '24': ['ADJF', 50.0], '27': ['ADJF', 50.0, 'ADVB', 50.0], '30': ['ADJF', 50.0], '58': ['ADJF', 66.66666666666666], '104': ['ADJF', 50.0], '115': ['ADJF', 50.0], '138': ['ADJF', 66.66666666666666], '145': ['ADJF', 66.66666666666666], '151': ['ADJF', 75.0], '157': ['ADJF', 60.0], '160': ['ADJF', 50.0, 'ADVB', 50.0], '164': ['ADJF', 50.0], '186'

### Рассмотрим некоторые нетипичные документы

In [14]:
for key in ['21', '56', '110', '130', '137', '157', '167', '230', '245', '275']:
    print(key, odd_docs[key], dict_corpus[key])

21 ['INFN', 100.0] ['проснулись']
56 ['NUMR', 5.263157894736842] ['обновления', 'мозга', 'админа', 'всё', 'друзья', 'числа', 'стартуем', 'догнать', 'семь', 'базовых', 'обновлений', 'улучшения', 'вашей', 'жизни', 'подписаться', 'подписался', 'получил', 'обновление', 'просто']
110 ['INFN', 60.0, 'PRCL', 20.0] ['ставьте', 'проснулись', 'всё', 'ещё', 'спите']
130 ['PRCL', 60.0] ['это', 'сон', 'это', 'лишь', 'сон']
137 ['PREP', 33.33333333333333] ['поняла', 'вокруг', 'сон']
157 ['ADJF', 60.0] ['доброй', 'ночи', 'ярких', 'осознанных', 'снов']
167 ['NOUN', 100.0] ['лайфхак']
230 ['NOUN', 15.151515151515152] ['произошло', 'странное', 'раньше', 'встречал', 'разных', 'сегодня', 'привязался', 'буквально', 'говорил', 'мной', 'словно', 'это', 'мудрец', 'вроде', 'должен', 'задавать', 'глупых', 'вопросов', 'скорей', 'те', 'помогут', 'развитие', 'почему', 'шёл', 'вёл', 'особо', 'спрашивал', 'узнал', 'это', 'вопрос', 'стену', 'заранее', 'благодарю']
245 ['NPRO', 11.11111111111111] ['вопрос', 'касательн

#### В подавляющем большинстве случаев нетипичная доля частей речи - целое число. Это обусловлено тем, что многие посты - это картинка с несколькими словами (документы 21, 110, 130, 137, 167). Есть интересные находки: в некоторых постах описаны сновидения, и из-за повествовательного характера текста доля существительных в документе 230 составила около 15%.